In [1]:
import sys
import numpy as np
from matplotlib import cm
import importlib as imp
sys.path.insert(0, '/home/hornung/projects/SimuJob')
import SimuJob as simj
imp.reload(simj)
import matplotlib.pyplot as plt
sys.path.insert(0, '/home/hornung/projects/soworm/pytools')

import simulator as sim
sys.path.insert(0, '/home/hornung/projects/pyworm')
import arrayworm as pwrm
from scipy.ndimage import uniform_filter
from scipy.ndimage.filters import gaussian_filter

In [2]:
import time

In [13]:
l1 =pwrm.Lattice(s1=50,s2=2,beta=5, mu3=0,mu8=-1)
for i in range(10):
    w= pwrm.Worm(l1)
    w.run()

In [4]:
def getlatet(lattice):
    r=10000000;
    cdict = {'u':1., 'd':-1., 's':0.}
    fundf = {1:1, 0:-1}
    res = np.array( [[cdict[lattice.sites[(x,y)].find_spin_at(r*t+1)] *
                      fundf[lattice.sites[(x,y)].isfundamental]
                          for t in range(lattice.tmax//r)] 
                               for x in range(lattice.s1) for y in range(lattice.s2)] )
    chargedensity = uniform_filter(res,size=(3,3), mode='constant')
    return chargedensity

def runwormgetlat2(lattice):

    r=100000000;
    cdict = {'u':1, 'd':2, 's':3}
    fundf = {1:1, 0:-1}
    res = np.array( [ [
                       cdict[lattice.sites[(x,y)].find_spin_at(1) ] *  fundf[lattice.sites[(x,y)].isfundamental]
                             for x in range(lattice.s1)] for y in range(lattice.s2)] )
    return np.transpose(res)

def getlatt3(lattice, t=1):
    cdict = {'u':1., 'd':-1., 's':0.}
    fundf = {1:1, 0:-1}
    res = np.array( [ [
                       cdict[lattice.sites[(x,y)].find_spin_at(t) ] *  fundf[lattice.sites[(x,y)].isfundamental]
                             for x in range(lattice.s1)] for y in range(lattice.s2)] )
    chargedensity = gaussian_filter(np.transpose(res),sigma=1, mode='constant')
    return chargedensity

def getlatt8(lattice):
    r=100000000;
    cdict = {'u':1./(2. * np.sqrt(3)), 'd':1./(2. * np.sqrt(3)), 's':-1./( np.sqrt(3))}
    fundf = {1:1, 0:-1}
    res = np.array( [ [
                       cdict[lattice.sites[(x,y)].find_spin_at(1) ] *  fundf[lattice.sites[(x,y)].isfundamental]
                             for x in range(lattice.s1)] for y in range(lattice.s2)] )
    chargedensity = gaussian_filter(np.transpose(res),sigma=1, mode='constant')
    return chargedensity


def getlatt3movie(lattice):
    r=10000000
    return [getlatt3(lattice, t) for t in range(1,lattice.tmax, r)]


In [98]:
ims = getlatt3movie(l1)

In [99]:
%gui qt

from pyqtgraph.Qt import QtCore, QtGui
import numpy as np
import pyqtgraph as pg
import pyqtgraph.ptime as ptime
win = pg.GraphicsLayoutWidget()
win.show()
view= win.addViewBox()
view.setAspectLocked(True)

img= pg.ImageItem(border='w',levels=[-0.5,0.5])
view.addItem(img)
colormap = cm.get_cmap("RdBu")  # cm.get_cmap("CMRmap")
colormap._init()
lut = (colormap._lut * 255).view(np.ndarray)  # Convert matplotlib colormap from 0-1 to 0 -255 for Qt
img.setLookupTable(lut)
img.setImage(np.random.random((100,100)))

i = len(ims)-1
def updateData():
    global i
    if i != 0:
        img.setImage(ims[i]) 
        img.setLevels([-0.5,0.5])
        i = i -1
        QtCore.QTimer.singleShot(5, updateData)
updateData()

In [50]:
img.setImage(im)

In [ ]:
len

In [9]:
%gui qt

from pyqtgraph.Qt import QtCore, QtGui
import numpy as np
import pyqtgraph as pg
import pyqtgraph.ptime as ptime

app = QtGui.QApplication([])

## Create window with GraphicsView widget
win = pg.GraphicsLayoutWidget()
win.show()  ## show widget alone in its own window
win.setWindowTitle('pyqtgraph example: ImageItem')
view = win.addViewBox()
view2 = win.addViewBox()
view3 = win.addViewBox()

win2 = pg.GraphicsLayoutWidget()
win2.show()
win2.setWindowTitle('ET stuff')
view4 = win2.addViewBox()

## lock the aspect ratio so pixels are always square
view.setAspectLocked(True)
view2.setAspectLocked(True)
view3.setAspectLocked(True)
#view4.setAspectLocked(True)

## Create image item
img = pg.ImageItem(border='w', levels=[-0.5,0.5])
imgt8 = pg.ImageItem(border='w', levels=[-0.5,0.5])

imget = pg.ImageItem(border='w', levels=[-0.5,0.5])
img2 = pg.ImageItem(border='w', levels=[-3,3])


plt = pg.PlotItem();
view.addItem(img)
view2.addItem(imgt8)
view3.addItem(img2)
view4.addItem(imget)
#view.addItem(plt)

## Create random image
data = runwormgetlat2(l1)

t3 = l1.t3();

updateTime = ptime.time()
fps = 10
lut = np.array([[0,0,125],[0,125,0],[125,0,0],[0,0,0],[255,0,0],[0,255,0],[0,0,255]])
img2.setLookupTable(lut)

colormap = cm.get_cmap("RdBu")  # cm.get_cmap("CMRmap")
colormap._init()
lut = (colormap._lut * 255).view(np.ndarray)  # Convert matplotlib colormap from 0-1 to 0 -255 for Qt
imget.setLookupTable(lut)

colormap = cm.get_cmap("RdBu")  # cm.get_cmap("CMRmap")
colormap._init()
lut = (colormap._lut * 255).view(np.ndarray)  # Convert matplotlib colormap from 0-1 to 0 -255 for Qt
img.setLookupTable(lut)

colormap = cm.get_cmap("PRGn")  # cm.get_cmap("CMRmap")
colormap._init()
lut = (colormap._lut * 255).view(np.ndarray)  # Convert matplotlib colormap from 0-1 to 0 -255 for Qt
imgt8.setLookupTable(lut)

def updateData():
    global img, data, updateTime, fps

    w= pwrm.Worm(l1)
    w.run()
    ## Display the data
    datat3 = getlatt3(l1)
    datat8 = getlatt8(l1)
    dataet = getlatet(l1)
    data2 = runwormgetlat2(l1)
    
    img.setImage(datat3)
    img.setLevels([-1,1])
    imgt8.setImage(datat8)
    imgt8.setLevels([-1,1])
    img2.setImage(data2)
    img2.setLevels([-3,3])
    imget.setImage(dataet)
    imget.setLevels([-1,1])

    QtCore.QTimer.singleShot(1, updateData)
    
    #print "%0.1f fps" % fps
    

updateData()

In [ ]:
runwormgetlat3(l1)

In [ ]:
1/(2*np.sqrt(3))

In [ ]:
max(1,2)